In [1]:
from torch.autograd import Variable
from torch.autograd import Function

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

In [1]:

import torch 
import torch.nn as nn
torch.manual_seed(seed=20200910)
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1=torch.nn.Sequential(  # 输入torch.Size([64, 1, 28, 28])
                torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
                torch.nn.ReLU(),  # 输出torch.Size([64, 64, 28, 28])
                torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),  # 输出torch.Size([64, 128, 28, 28])
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(stride=2,kernel_size=2)  # 输出torch.Size([64, 128, 14, 14])
        )

        self.dense=torch.nn.Sequential(  # 输入torch.Size([64, 14*14*128])
                    torch.nn.Linear(14*14*128,1024),  # 输出torch.Size([64, 1024])
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.5),
                    torch.nn.Linear(1024,10)  # 输出torch.Size([64, 10])        
        )
        self.layer4cxq1 = torch.nn.Conv2d(2,33,4,4)
        self.layer4cxq2 = torch.nn.ReLU()
        self.layer4cxq3 = torch.nn.MaxPool2d(stride=2,kernel_size=2)
        self.layer4cxq4 = torch.nn.Linear(14*14*128,1024)
        self.layer4cxq5 = torch.nn.Dropout(p=0.8)
        self.attribute4cxq = nn.Parameter(torch.tensor(20200910.0))
        self.attribute4lzq = nn.Parameter(torch.tensor([2.0,3.0,4.0,5.0]))    
        self.attribute4hh = nn.Parameter(torch.randn(3,4,5,6))
        self.attribute4wyf = nn.Parameter(torch.randn(7,8,9,10))

    def forward(self,x):  # torch.Size([64, 1, 28, 28])
        x = self.conv1(x)  # 输出torch.Size([64, 128, 14, 14])
        x = x.view(-1,14*14*128)  # torch.Size([64, 14*14*128])
        x = self.dense(x)  # 输出torch.Size([64, 10])
        return x

print('cuda(GPU)是否可用:',torch.cuda.is_available())
print('torch的版本:',torch.__version__)

model = Model() #.cuda()

model_1 = model.eval()
print(id(model))
print(id(model_1))
print(id(model)==id(model_1))


cuda(GPU)是否可用: False
torch的版本: 1.12.1+cu102
140221960903744
140221960903744
True


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
class Classifier(nn.Module):
    def __init__(self, f_dims=512):
        super(Classifier, self).__init__()
        #first fc
        self.fc1 = nn.Linear(f_dims, f_dims//4)
        self.bn1_fc = nn.BatchNorm1d(f_dims//4)
        #second fc
        self.fc2 = nn.Linear(f_dims//4, 2) 
    def forward(self, x):
        x = F.relu(self.bn1_fc(self.fc1(x)))
        x = self.fc2(x)
        return x

if __name__=='__main__':
    classifier = Classifier(f_dims=96)
    B=10
    input=torch.randn(B,96)
    output=classifier(input) #(B,2)
    di_s_label=torch.zeros(output.shape[0]).long() #(B,)
    crossentropyloss=nn.CrossEntropyLoss()

    virtual_di_loss = crossentropyloss(output, di_s_label)
    print(virtual_di_loss)


tensor(0.7865, grad_fn=<NllLossBackward0>)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Discriminator(nn.Module):
    def __init__(self, features,version=0):
        self.layer_num = len(features)-1
        super(Discriminator, self).__init__()

        self.fc_layers = nn.ModuleList([])
        for inx in range(self.layer_num):
            self.fc_layers.append(nn.Conv1d(features[inx], features[inx+1], kernel_size=1, stride=1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)

        self.final_layer = nn.Sequential(
                    nn.Linear(features[-1], 128),
                    nn.LeakyReLU(negative_slope=0.2),
                    nn.Linear(128, 64),
                    nn.LeakyReLU(negative_slope=0.2),
                    nn.Linear(64, 1),
                    nn.Sigmoid())

    def forward(self, f):
        
        feat = f.transpose(1,2)
        vertex_num = feat.size(2)

        for inx in range(self.layer_num):
            feat = self.fc_layers[inx](feat)
            feat = self.leaky_relu(feat)
        out1 = F.max_pool1d(input=feat, kernel_size=vertex_num).squeeze(-1)
        out = self.final_layer(out1) # (B, 1)
        return out,out1

features=[3, 64,  128, 256, 256, 512] # Features for discriminator
D = Discriminator(features=features)
pcd= torch.randn(10,2048,3)
feat=D(pcd)
print(feat[0].shape)


torch.Size([10, 1])


In [1]:
import torch
pcd=torch.randn(2048,3)
pcd.unsqueeze(0).shape

torch.Size([1, 2048, 3])

In [6]:
import torch
a=torch.randn(3,3)
# b=a
b.copy_(a)
print(id(a))
print(id(b))

140038212241744
140038856096560
